# Cambiamenti

- Provare ad usare un solo hidden layer
- Provare a cambiare funzione d'attivazione dell'output layer da lineare a ReLu
- Cambiare l'ultimo grafico


In [1]:
import numpy as np
import pandas as pd
from re import sub
from time import time

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras_tqdm import TQDMNotebookCallback as ktqdm
from keras.utils import normalize
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2

from tensorflow.nn import relu, softmax

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, Normalizer
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score

import scipy.stats as st

import seaborn as sns

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
df = pd.read_csv('fifa19.csv')

In [3]:
df.drop(columns=['Unnamed: 0', 'ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Preferred Foot',
                 'Body Type', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until'],inplace=True)

## Obiettivo: predire valore dei giocatori

Pre-processing: convertire value, wage e release clause da string a float

In [4]:
curs=["Release Clause", "Value", "Wage"]

for cur in curs:
    def curr2val(x):
        x = str(x).replace('€', '')
        if 'K' in x: x = float(str(x).replace('K', '')) * 1000
        else: x = float(str(x).replace('M', '')) * 1000000
        return x
    df[cur] = df[cur].apply(curr2val)
    

Individuare eventuali outlier nella colonna value

In [5]:
def detect_outlier(data, threshold = 3):
    outliers=[]
    mean = np.mean(data)
    std = np.std(data)
    
    for y in data:
        score= (y - mean) / std 
        if np.abs(score) > threshold:
            outliers.append(y)
    return outliers

In [6]:
min_out = min(detect_outlier(df['Value'], threshold = 2))

df = df[df['Value'] < min_out] 
df = df[df['Value'] > 0]

Conversione in interi per le altre label

In [7]:
cols=["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW","LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM","CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"]
for col in cols:
    df[col] = df[col].str[:-2]
    df[col] = df[col].astype(float)

In [8]:
df['Height'] = df['Height'].str.replace("'",'.')
df['Height'] = df['Height'].astype(float)

df['Weight'] = df['Weight'].str[:-3]
df['Weight'] = df['Weight'].astype(float)

Calcolo correlazione tra i valori per scegliere colonne significative

In [9]:
df_corr = df.corr()

# fig = plt.figure(figsize=(50,20))
# ax = fig.add_subplot(111)
# cax = ax.matshow(df_corr,cmap='coolwarm', vmin=-1, vmax=1)
# fig.colorbar(cax)

# ticks = np.arange(0,len(df_corr.columns),1)
# ax.set_xticks(ticks)
# ax.set_xticklabels(df_corr.columns)
# plt.xticks(rotation=90)
# ax.set_yticks(ticks)
# ax.set_yticklabels(df_corr.columns)

# plt.show()

In [10]:
labels = []
for label in df_corr:
#     if df_corr['Value'][label] < 0 or df_corr['Value'][label] > 0.5: labels.append(label)
    if df_corr['Value'][label] > 0.6: labels.append(label)
        
df_flt = df[labels]        
df_flt.head()      

,Overall,Potential,Value,Wage,Reactions,Release Clause
41,88,88,4000000.0,77000.0,79.0,7400000.0
102,85,85,9000000.0,38000.0,85.0,15300000.0
108,85,85,9000000.0,57000.0,83.0,17100000.0
152,84,84,4200000.0,95000.0,80.0,6900000.0
201,83,83,13000000.0,70000.0,78.0,24700000.0


Mescolo le righe del dataset

In [11]:
df_flt = df_flt.sample(frac=1)

In [12]:
y = df_flt.loc[:, ['Value']]
X = df_flt.drop(columns='Value')

Sostiuisco eventuali valori nan con la media della colonna

In [13]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X)
X_full = imputer.transform(X)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y)
y_full = imputer.transform(y)

Scalo i valori, sia per i caratteri che per il target

In [14]:
# scaler = RobustScaler()
# scaler = scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# X_train_scaled = preprocessing.scale(X_train)
scaler = StandardScaler().fit(X_full)
X_scaled = scaler.transform(X_full)

scaler_train = StandardScaler().fit(y_full)
y_scaled = scaler_train.transform(y_full) 
# X_train_scaled, X_test_scaled

In [15]:
def coeff_determination(y_test, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_test-y_pred ))
    SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

In [16]:
def build_nn(neurons):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(256, input_dim = neurons, activation='relu'))#, kernel_regularizer=l2(0.001)))
#     model.add(Dropout(0.01))
    
    # Adding the first hidden layer
    model.add(Dense(256, activation = 'relu'))#, kernel_regularizer=l2(0.001)))
#     model.add(Dropout(0.01))
    
    # Adding the second hidden layer
#     model.add(Dense(256, activation = 'relu'))#, kernel_regularizer=l2(0.001)))
    
    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))
    
    return model

In [17]:
ts_board = TensorBoard(log_dir='value_predictions_v3/{}'.format('kfold_' + str(time())))

In [18]:
def mean_abs_error(prediction, target):
    res = 0
    tot = 0
    for i in range(len(target)):
        if target[i][0] != 0 and not np.isnan(prediction[i][0]):
#             print(prediction[i][0], target[i][0])
            res += abs(prediction[i][0] - target[i][0])
            tot += 1
    print(tot, len(target), len(prediction))
    return round(res / tot, 2)

# Cross-validation
### K-Fold

In [19]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
kfold = KFold(n_splits=5, random_state=42, shuffle=False)

In [20]:
scores = []

nn = build_nn(X_scaled.shape[1])

opt = Adam(lr=0.001, beta_1=0.75, beta_2=0.999, epsilon=0.001, decay=0.0, amsgrad=False)
# opt = SGD(lr=0.01, momentum=0.9)
nn.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
for train, test in kfold.split(X_scaled):
    X_train_round, X_test_round, y_train_round, y_test_round = X_scaled[train], X_scaled[test], y_scaled[train], y_scaled[test]
        
    nn.fit(X_train_round, y_train_round, batch_size=125, epochs=50, shuffle=True)
    
    # predict
    res = nn.predict(X_test_round)
    print(round(r2_score(y_test_round, res), 4))
    
    # evaluate
    score = nn.evaluate(X_test_round, y_test_round, verbose=1)
    print("loss: %.4f, acc: %.4f%%" % (score[0], score[1]*100))
    
    scores.append(score[1] * 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
13928/13928 [==============================] - 1s 39us/step - loss: 0.1516 - coeff_determination: 0.8645
Epoch 2/50
13928/13928 [==============================] - 0s 21us/step - loss: 0.0954 - coeff_determination: 0.9356
Epoch 3/50
13928/13928 [==============================] - 0s 21us/step - loss: 0.0920 - coeff_determination: 0.9412
Epoch 4/50
13928/13928 [==============================] - 0s 21us/step - loss: 0.0887 - coeff_determination: 0.9430
Epoch 5/50
13928/13928 [==============================] - 0s 21us/step - loss: 0.0877 - coeff_determination: 0.9467
Epoch 6/50
13928/13928 [==============================] - 0s 21us/step - loss: 0.0840 - coeff_determination: 0.9503
Epoch 7/50
13928/13928 [==============================] - 0s 21us/step - loss: 0.0820 - coeff_determination: 0.9536
Epoch 8/50
13928/13928 [==============================] - 0s 21us/step - loss: 0.0790 - coeff_determination: 0.9572
Epoch 9/50
13928/13928 [

13929/13929 [==============================] - 0s 20us/step - loss: 0.0562 - coeff_determination: 0.9804
Epoch 19/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0561 - coeff_determination: 0.9806
Epoch 20/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0556 - coeff_determination: 0.9818
Epoch 21/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0565 - coeff_determination: 0.9799
Epoch 22/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0556 - coeff_determination: 0.9807
Epoch 23/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0552 - coeff_determination: 0.9816
Epoch 24/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0561 - coeff_determination: 0.9802
Epoch 25/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0552 - coeff_determination: 0.9805
Epoch 26/50
13929/13929 [==============================] - 0s 19us/step - lo

13929/13929 [==============================] - 0s 20us/step - loss: 0.0521 - coeff_determination: 0.9848
Epoch 38/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0523 - coeff_determination: 0.9837
Epoch 39/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0520 - coeff_determination: 0.9838
Epoch 40/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0525 - coeff_determination: 0.9840
Epoch 41/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0521 - coeff_determination: 0.9840
Epoch 42/50
13929/13929 [==============================] - 0s 20us/step - loss: 0.0522 - coeff_determination: 0.9837
Epoch 43/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0515 - coeff_determination: 0.9851
Epoch 44/50
13929/13929 [==============================] - 0s 20us/step - loss: 0.0515 - coeff_determination: 0.9848
Epoch 45/50
13929/13929 [==============================] - 0s 19us/step - lo

13929/13929 [==============================] - 0s 19us/step - loss: 0.0501 - coeff_determination: 0.9861
Epoch 6/50
13929/13929 [==============================] - 0s 20us/step - loss: 0.0503 - coeff_determination: 0.9854
Epoch 7/50
13929/13929 [==============================] - 0s 20us/step - loss: 0.0497 - coeff_determination: 0.9861
Epoch 8/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0502 - coeff_determination: 0.9855
Epoch 9/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0503 - coeff_determination: 0.9855
Epoch 10/50
13929/13929 [==============================] - 0s 20us/step - loss: 0.0499 - coeff_determination: 0.9856
Epoch 11/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0500 - coeff_determination: 0.9856
Epoch 12/50
13929/13929 [==============================] - 0s 19us/step - loss: 0.0496 - coeff_determination: 0.9861
Epoch 13/50
13929/13929 [==============================] - 0s 19us/step - loss: 

In [22]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores)))

98.04% (+/- 0.31%)


In [23]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_full, test_size = 0.15, random_state = 42)

In [24]:
res = nn.predict(X_val)
res = scaler_train.inverse_transform(res)
mean_abs_error(res, y_val), max(y_val), round(r2_score(y_val, res), 4)

2612 2612 2612


(118511.75, array([13500000.]), 0.9877)